# Imbalanced classification: credit card fraud detection

## Train a classification model on data with highly imbalanced classes.

In [1]:
### 1. Read Data

In [3]:
import csv
import numpy as np

# Data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
# Download it to your local dir
fname = "C://Users//bokhy//Desktop//Python//Python-Projects//Pytorch//creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


### 2. Train/Test Split

In [4]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


### 3. Analyze class imbalance in the Response variable

In [7]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


In [8]:
### Normalize the data using training set statistics
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

### 4. Create a Model

In [9]:
# Binary classification model
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
__________________________________________________

### 5. Model Training

In [11]:
# Train the model with class_weight argument
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

#save model for each epoch by making a callback
#callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]

class_weight = {0: weight_for_0, 1: weight_for_1}

# Model fitting by adding test(validation)data
model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    #callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight, ## important!
)

Train on 227846 samples, validate on 56961 samples
Epoch 1/30
227846/227846 - 5s - loss: 6.2358e-07 - fn: 17.0000 - fp: 5062.0000 - tn: 222367.0000 - tp: 400.0000 - precision: 0.0732 - recall: 0.9592 - val_loss: 1.2858e-06 - val_fn: 8.0000 - val_fp: 621.0000 - val_tn: 56265.0000 - val_tp: 67.0000 - val_precision: 0.0974 - val_recall: 0.8933
Epoch 2/30
227846/227846 - 3s - loss: 5.2353e-07 - fn: 10.0000 - fp: 5086.0000 - tn: 222343.0000 - tp: 407.0000 - precision: 0.0741 - recall: 0.9760 - val_loss: 2.2980e-06 - val_fn: 9.0000 - val_fp: 543.0000 - val_tn: 56343.0000 - val_tp: 66.0000 - val_precision: 0.1084 - val_recall: 0.8800
Epoch 3/30
227846/227846 - 4s - loss: 1.0554e-06 - fn: 19.0000 - fp: 7910.0000 - tn: 219519.0000 - tp: 398.0000 - precision: 0.0479 - recall: 0.9544 - val_loss: 1.8638e-06 - val_fn: 5.0000 - val_fp: 4612.0000 - val_tn: 52274.0000 - val_tp: 70.0000 - val_precision: 0.0150 - val_recall: 0.9333
Epoch 4/30
227846/227846 - 5s - loss: 8.4623e-07 - fn: 19.0000 - fp: 598

In [13]:
loss, acc = model.evaluate(val_features)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

32/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 779us/sample


IndexError: list index out of range

# Conclusions
## At the end of training, out of 56,961 validation transactions, we are:

- >Correctly identifying 64 of them as fraudulent
- >Missing 11 fraudulent transactions at the cost of incorrectly flagging 450 legitimate transactions